In [23]:
import cryptocompare
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime, timedelta
plt.rcParams["figure.figsize"] = (18,7)

In [31]:
# how much data you want
limit_data = 2000 #(hours)
# What you wanna trade
coin = 'ENJ'
# Base currency 
currency = 'BTC'

In [32]:
data = cryptocompare.get_historical_price_hour(coin, curr=currency, limit= limit_data, exchange='Binance')

In [33]:
# mean = []
time = []
high = []
low = []
open1 = []
close1 = []
volume = []
for i in range(len(data)):
    hi = data[i]['high']
    lo = data[i]['low']
    op = data[i]['open']
    cl = data[i]['close']
    vl = data[i]['volumefrom']
#     mean.append((hi+lo)/2)
    time.append(i)
    high.append(hi)
    low.append(lo)
    open1.append(op)
    close1.append(cl)
    volume.append(vl)
#df = pd.DataFrame(list(zip(pd.Series(pd.date_range('2000', freq='H', periods=limit_data)),high,low,open1,close1,volume,mean)), columns =['Date','high','low','open','close','volumefrom', 'mean']) 

time_delta = timedelta(hours=limit_data, minutes=0, seconds=0, microseconds=0) # minus hours 
date_at_start = datetime.now() - time_delta
days = pd.date_range(date_at_start, date_at_start + timedelta(limit_data/24), freq='H') # if freq is in hours then divide limit_data by 24

df = pd.DataFrame({'Date': days,'Open':open1, 'High': high, 'Low' : low, 'Close': close1, 'Volume' : volume})
df = df.set_index('Date')

In [34]:
df

,Open,High,Low,Close,Volume
Date,,,,,
2020-10-15 14:12:59.586447,0.000014,0.000015,0.000014,0.000015,149310
2020-10-15 15:12:59.586447,0.000015,0.000015,0.000014,0.000014,86714
2020-10-15 16:12:59.586447,0.000014,0.000014,0.000014,0.000014,101505
2020-10-15 17:12:59.586447,0.000014,0.000014,0.000014,0.000014,398878
2020-10-15 18:12:59.586447,0.000014,0.000014,0.000014,0.000014,151979
...,...,...,...,...,...
2021-01-06 18:12:59.586447,0.000006,0.000006,0.000006,0.000006,758702
2021-01-06 19:12:59.586447,0.000006,0.000006,0.000006,0.000006,594056
2021-01-06 20:12:59.586447,0.000006,0.000006,0.000006,0.000006,1466136


In [37]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA

class SmaCross(Strategy):
    def init(self):
        price_close = self.data.Close
        self.ma1 = self.I(SMA, price_close, 7)    #7 and 39 gives 54% return SUSHI hourly chart
        self.ma2 = self.I(SMA, price_close, 35)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()



bt = Backtest(df, SmaCross, commission=.002,
              exclusive_orders=True)
stats = bt.run()
print(type(stats))
bt.plot()

<class 'backtesting.backtesting.Backtest._Stats'>


<ipython-input-37-44b9f65b8d74>:24: UserWarning: 'Can't superimpose OHLC data with rule 'None'(index datetime resolution: 'microsecond'). Skipping.
  bt.plot()
